In [1]:
import os

In [2]:
%pwd

'/Users/shrikanthnallapati/Projects/Chicken_Disease_Classification/ImageClassification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/shrikanthnallapati/Projects/Chicken_Disease_Classification/ImageClassification'

# Now update the entity which is the return type of the function either inbuild type or custom type (eg - ConfigBox in utils/common.py)

In [5]:
# Entity _ which return type should the function return.
#Data Ingestion related config

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file : Path
    unzip_dir : Path

# In order to return the above class, we need to read yaml (config.yaml) file. 
# so define the path for config and params files in the constants file 
# Now we will be able to pull the info via contant file 

In [12]:
# * means everything 
from src_cnnClassifier.constants import * 
from src_cnnClassifier.utils.common import read_yaml, create_directories

# Now update Configuration manager in src folder 

In [20]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion # storing the data_ingestion config into into variable config

          create_directories([config.root_dir]) # create root directory for the variable config 

          # Returning the data type as defined in teh DataIngestionCofig in the entity or above step #5.
          # reading one by one from the config variable and storing them to assigned variables like root_dir, source_URL etc. 

          data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          )

          return data_ingestion_config
    

# update the componets 

In [16]:
import os
import urllib.request as request
import zipfile
from src_cnnClassifier import logger
from src_cnnClassifier.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

# download the file and unzip it 
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory 
        Function returns None 
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            


# create the Pipeline

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e



NameError: name 'ConfigurationManager' is not defined